In [ ]:
# Imports

import pandas as pd
import sqlite3
import shutil
import tempfile
from pathlib import Path

## Make a temporary copy of the Messages database

In [ ]:
src = Path("~/Library/Messages/chat.db").expanduser()
tmp_db = Path(tempfile.gettempdir()) / "chat_copy.db"
try:
    shutil.copy2(src, tmp_db)  # requires read permission on Messages.db
except PermissionError as e:
    raise RuntimeError(
        f"Cannot read {src}. Grant Full Disk Access to the Python/Jupyter process (System Settings → Privacy & Security → Full Disk Access) and re-run."
    ) from e

In [ ]:
print(tmp_db)

## Connect to the temporary database

In [ ]:
con = sqlite3.connect(tmp_db)

In [ ]:
cur = con.cursor()
res = cur.execute("SELECT sql FROM sqlite_master WHERE type='table'")
for r in res.fetchall():
    print(r)
# res = cur.execute("SELECT * FROM message LIMIT 1")
# print(res.fetchall())
res.close()
cur.close()

In [ ]:
messages_df = pd.read_sql("SELECT * FROM message", con)


In [ ]:
desc = messages_df.describe()
for col in desc:
    print(desc[col])
    print()

In [ ]:
# columns = "text, service_center, subject, country, attributedBody, service, account, account_guid, error, date, date_read, date_delivered, is_delivered, is_from_me, is_read, is_sent, cache_roomnames, was_data_detected, is_audio_message, is_played, date_played, group_title, group_action_type, message_source, associated_message_guid, associated_message_type, payload_data, destination_caller_id, reply_to_guid, thread_originator_guid, thread_originator_part, was_delivered_quietly, date_edited, part_count, associated_message_emoji"
columns = "ROWID, text, service_center, subject, country, attributedBody, service, date, is_from_me, group_title, part_count, handle_id"
messages_df = pd.read_sql(f"SELECT {columns} FROM message WHERE text IS NOT NULL AND error = 0 ORDER BY date DESC LIMIT 10", con)

# Convert the date column from Apple Absolute timestamp to a human-readable format
messages_df['date'] = pd.to_datetime(messages_df['date'] + 978307200000000000, unit='ns')

# Convert the bytes attributedBody to plain text
def bytes_to_text(bytes_obj):
    return bytes_obj.decode('utf-8', errors='ignore')
messages_df['attributedBody'] = messages_df['attributedBody'].apply(bytes_to_text)

print(", ".join(list(messages_df.columns)))
print(messages_df['attributedBody'].dtype)
print(str(messages_df['attributedBody'][0]))

# for col in messages_df.columns:
#     print(f"{col}: {messages_df[col]}")
messages_df


In [ ]:
chats_df = pd.read_sql("SELECT * FROM chat", con)
", ".join(list(chats_df.columns))

In [ ]:
desc = chats_df.describe()
for col in desc:
    print(desc[col])
    print()

In [ ]:
columns = "ROWID, properties, chat_identifier, service_name, room_name, account_login, display_name, group_id, engram_id, original_group_id, last_read_message_timestamp"
chats_df = pd.read_sql(f"SELECT {columns} FROM chat ORDER BY last_read_message_timestamp DESC LIMIT 10", con)

# # Convert the date column from nanoseconds since epoch to a human-readable format
chats_df['last_read_message_timestamp'] = pd.to_datetime(chats_df['last_read_message_timestamp'] + 978307200000000000, unit='ns')

def bytes_to_text(bytes_obj):
    if bytes_obj is None:
        return None
    return bytes_obj.decode('utf-8', errors='ignore')
chats_df['properties'] = chats_df['properties'].apply(bytes_to_text)

chats_df
# for row in chats_df.itertuples():
#     print(row.properties, row.chat_identifier, row.last_read_message_timestamp)

In [ ]:
chat_id = input("Enter chat ROWID to get messages: ")
chat_messages_df = pd.read_sql(f"SELECT * FROM chat_message_join JOIN message ON chat_message_join.message_id = message.ROWID WHERE chat_id = {chat_id} AND text IS NOT NULL ORDER BY chat_message_join.message_date DESC LIMIT 100", con)

# Convert the date column from Apple Absolute timestamp to a human-readable format
chat_messages_df['date'] = pd.to_datetime(chat_messages_df['date'] + 978307200000000000, unit='ns')

for message in chat_messages_df.itertuples():
    print(message.text, message.date)